In [ ]:
import glob
import tgt
import os
import nltk
import cmudict
import conll
import re

def dico2FeatureString(dico):
    """Transforms a dictionary of Conll features into a string"""
    feature = ["=".join([key, dico[key]]) for key in dico]
    feature = "|".join(feature)
    return feature

def build_feature_dico(misc_features_string):
    """Turns a string of CONLL features into a callable dictionary"""
    feature_dico = {}
    for feature in misc_features_string.split("|"):
        key, value = feature.split("=")
        feature_dico[key] = value
    return feature_dico

def confirm_alignment(ref_interval, target_interval):
    """Takes two pitchtier intervals as entry and returns whether or not there is a temporal overlap, as well as the nature of that overlap."""
    
    left_overlap = False
    right_overlap = False

    
    #print(ref_interval, target_interval)
    if ref_interval.text == "":
        # print("empty string") # debug
        return False, left_overlap, right_overlap  # disregard pauses

    if round(target_interval.end_time - ref_interval.start_time,3) <= 0.001:
        #print("A") # debug
        return False, left_overlap, right_overlap  
    
    elif round(target_interval.start_time - ref_interval.end_time,3) >= -0.001:
        #print("B") # debug 
        return False, left_overlap, right_overlap  
    
    if  round(ref_interval.start_time - target_interval.start_time, 3) >= 0.01:
        #print("C", str(ref_interval.start_time - target_interval.start_time)) # debug
        left_overlap = True
    if round(target_interval.end_time - ref_interval.end_time, 3)  >= 0.01:
        #print("D") # debug
        right_overlap = True

    return True, left_overlap, right_overlap 



def vowel_final(word_string, dico):
    """Takes as entry an orthographic word string and a pronunciation dictionary, and confirms whether or not the canonical pronunciation of a that word ends in a vowel"""
    vowels="aeiouAEIOU" 
    if word_string == "":
        return False
    try:
        phonemes = pronunciation[word_string.lower()][0]
        if phonemes[-1][0] in vowels:
            return True
    except:
        if word_string[-1] in vowels:
            return True
    return False
        
def extractProsodicAnnotation(textgrid, label_tier, label_tier2, token_tier, word_text_tier, syllable_transcription_tier):
    """
    Extracts prosodic information from a .TextGrid file containing SLAM annoations and returns a dictionary of annotations.

    Takes as entry:
    * A textgrid file containing tiers corresponding to the following elements.
    * Name of the tier containing phonetic transcriptions of syllables (syllable_transcription_tier)
    * Names of two tiers containing global (label_tier) and local (label_tier2) SLAM annotations of syllables 
    * Name of the tier containing a token-level alignment and numeric ID in format 2:13 (utterance two, token 13) (token_tier)
    * Name of tier containing orthographic transcription of token (word_text_tier)
    """

    pronunciation = cmudict.dict()
    dico = {}
    
    try: textgrid_object = tgt.read_textgrid(textgrid, include_empty_intervals=True, encoding="utf-8")
    except: textgrid_object = tgt.read_textgrid(textgrid, include_empty_intervals=True, encoding="utf-16")
    label_obj = textgrid_object.get_tier_by_name(label_tier)
    label_obj2 = textgrid_object.get_tier_by_name(label_tier2)
    token_object = textgrid_object.get_tier_by_name(token_tier)  
    word_text_object = textgrid_object.get_tier_by_name(word_text_tier)
    syllable_transcription_object = textgrid_object.get_tier_by_name(syllable_transcription_tier)
    
    i=0
        
    for index, token in enumerate(token_object): 
        
        skip_features = False
        
        alignment_found = False
        #alignment = False
        syl_number = 1
        
        if not token.text: continue
        
        dico[token.text] = {}
        dico[token.text]["AlignBegin"] = str(int(token.start_time*1000))
        dico[token.text]["AlignEnd"] = str(int(token.end_time*1000))
        
        ### Debugging/holdover from earlier version ###
        #dico[token.text]["LeftOverlap"] = "False"
        #dico[token.text]["RightOverlap"] = "False"
        
        alignment, left_overlap, right_overlap = confirm_alignment(token, label_obj[i])
        #print(alignment, left_overlap, right_overlap) # debug
        #print(token) # debug 
        while not alignment:
            i+=1 
            #print(i) # debug 
            alignment, left_overlap, right_overlap = confirm_alignment(token, label_obj[i])
            #print(alignment, left_overlap, right_overlap) # debug

        
        while alignment:
            skip_features = False
            if left_overlap:
                 ### Debug ###
                #print(dico[token.text])
                #print(dico[token_object[index-1].text])
                #print(token.text, token_object[index-1].text)
                #print(token, label_obj[i])
                #print(token_object[index-1].text)
                #print(left_overlap)

                try: 
                    if vowel_final(word_text_object[index-1].text, pronunciation): 
                        dico[token.text]["Syl"+str(syl_number)] = "FUSED"
                        skip_features = True

                    ### Handles rare cases of triple fusions betweeen syllables

                    elif dico[token_object[index-1].text]["Syl1"] == "FUSED" and "Syl2" not in dico[token_object[index-1].text]:
                        dico[token.text]["Syl"+str(syl_number)] = "FUSED"
                        skip_features = True

                    elif "Syl1AlignBegin" in dico[token_object[index-1].text] and str(dico[token_object[index-1].text]['Syl1AlignBegin']) == str(int(label_obj[i].start_time*1000)):
                        dico[token.text]["Syl"+str(syl_number)] = "FUSED"
                        skip_features = True

                    else:
                        dico[token.text]["Syl"+str(syl_number)+"ExternalOnset"] = "True"
                except:
                    print("problem on {}, confirm output".format(label_obj[i]))

                #dico[token.text]["LeftOverlap"] = "True"
                
            if right_overlap:
                if vowel_final(word_text_object[index].text, pronunciation) != True and label_obj[i].start_time > token.start_time:  
                    skip_features = True

                #dico[token.text]["RightOverlap"] = "True"
                
            if skip_features == False:
            
                if label_obj[i].text:
                    dico[token.text]["Syl"+str(syl_number)+"Glo"] = label_obj[i].text
                    dico[token.text]["Syl"+str(syl_number)+"Loc"] = label_obj2[i].text
                    dico[token.text]["Syl"+str(syl_number)+"Duration"] = str(int((label_obj[i].end_time - label_obj[i].start_time)*1000))
                    
                    feature_dico = contourToFeatures(label_obj[i].text, "Glo")
                    feature_dico.update(contourToFeatures(label_obj2[i].text, "Loc"))

                    for feature in feature_dico.keys():
                        dico[token.text]["Syl"+str(syl_number)+feature] = feature_dico[feature]
                    dico[token.text]["SyllableCount"] = str(syl_number)

                    #dico[token.text]["SylStart"] = label_obj[i].start_time

                else: 
                    dico[token.text]["Syl"+str(syl_number)+"Glo"] = "X"
                    dico[token.text]["Syl"+str(syl_number)+"Loc"] = "X"
                    #dico[token.text]["SylStart"] = label_obj[i].start_time
                    
                
                dico[token.text]["Syl"+str(syl_number)+"AlignBegin"] = str(int(label_obj[i].start_time*1000))
                dico[token.text]["Syl"+str(syl_number)+"AlignEnd"] = str(int(label_obj[i].end_time*1000))

                dico[token.text]["Syl"+str(syl_number)] = syllable_transcription_object[i].text


            syl_number+=1
            i+=1
            if i == len(label_obj):
                break
                
            alignment, left_overlap, right_overlap = confirm_alignment(token, label_obj[i])
            #print(alignment, left_overlap, right_overlap)

        
        i-=1
        

                
    return dico

def translate_code(code):
    """Takes in entry a textual SLAM label and converts it into a list of numeric values to facilitate extraction of features"""

    translated_code = []
    for letter in code[0:2]:
        if letter == "L":
            translated_code.append(1)
        elif letter == "l":
            translated_code.append(2)
        elif letter == "m":
            translated_code.append(3)
        elif letter == "h":
            translated_code.append(4)
        elif letter == "H":
            translated_code.append(5)

    if len(code) > 2:
        if code[2] == "L":
            translated_code.append((1, int(code[3])))
        elif code[2] == "l":
            translated_code.append((2, int(code[3])))
        elif code[2] == "m":
            translated_code.append((3, int(code[3])))
        elif code[2] == "h":
            translated_code.append((4, int(code[3])))
        elif code[2] == "H":
            translated_code.append((5, int(code[3])))

    return translated_code
            
def contourToFeatures(contour_label, suffix=""):
    """Takes as entry a textual SLAM label and returns a set of categorical prosodic features describing the label."""
    
    dico = {}

    code = translate_code(contour_label)

    if code[0] == code[1]:
        dico["Slope"+suffix] = "Flat"
    elif code[0] < code[1]:
        dico["Slope"+suffix] = "Rise"
    elif code[0] > code[1]:
        dico["Slope"+suffix] = "Fall"

    if len(code) == 2:
        height = (code[0] + code[1]) / 2
    if len(code) == 3:
        height = (code[0] + code[1] + code[2][0]) / 3

    if height >= 3.5:
        dico["AvgHeight"+suffix] = "H"
    elif height < 2.5:
        dico["AvgHeight"+suffix] = "L"
    else: 
        dico["AvgHeight"+suffix] = "M"

    amplitude = abs(code[0] - code[1])
    if amplitude <= 1:
        dico["PitchRange"+suffix] = "L"
    elif amplitude >= 3:
        dico["PitchRange"+suffix] = "H"
    else: 
        dico["PitchRange"+suffix] = "M"
        
    return dico

# Definition of parameters

In [ ]:
pronunciation = cmudict.dict() # Pronunciation dictionary: to modify according to the language chosen
syl_tier = "SyllablesStyleGlo" # Tier containing global contour
syl_tier2 = "SyllablesStyleLoc" # Tier containing local contour
word_tier = "Word-ID" # Tier containing numeric ID for token 
word_text_tier = "Word-Text" # Tier containing text token text 
syllable_text_tier = "Syllables" # Tier containing syllabic transcriptions 
slam_files = glob.glob("SLAM_output/*.TextGrid") # Folder containing SLAM labels in TextGrid format
conll_infiles = glob.glob("CONLL_files/*.conllu") # Folder containing CONLLU files to which prosodic information will be added
conll_outfolder = "CONLL_outfiles/"

# If running this script a second time, useful for renaming features 
feature_rename_dict = {"OldFeatname" : "NewFeatname"}

# Fill CONLLU files

In [ ]:
for slam_file in sorted(slam_files):
    basename = os.path.basename(slam_file)[:-9]
    print("treating", basename)
    annotations = extractProsodicAnnotation(slam_file, syl_tier, syl_tier2, word_tier, word_text_tier, syllable_text_tier)
    for infile in conll_infiles:
        
        if os.path.basename(infile)[:len(basename)] == basename:
            trees=conll.conllFile2trees(infile)
            for treei, tree in enumerate(trees):
                for token in tree:
                    identifier = str(treei+1)+":"+str(token)
                    misc_features = tree[token]['misc']
                    feature_dico = build_feature_dico(misc_features)
                    
                    
                    features_to_delete = []
                    
                    new_feature_dico = {}
                    for feature in feature_dico.keys():
                        #print(feature)
                        if re.match("Syl[0-9].*", feature):
                            if "Amplitude" not in feature:
                                features_to_delete.append(feature)
                            #print(features_to_delete)
                            
                        #print(feature)
                        if feature[4:] in feature_rename_dict.keys():
                            newfeat = feature.replace(feature[4:], feature_rename_dict[feature[4:]])
                            #print(feature, newfeat)
                            newval = feature_dico[feature]
                            new_feature_dico[newfeat] = newval
                            features_to_delete.append(feature)
                            #print(feature_dico)

                        """"   
                        if feature == "F0Enonce":
                            newfeat = feature.replace(feature, feature_rename_dict[feature])
                            #print(feature, newfeat)
                            newval = feature_dico[feature]
                            new_feature_dico[newfeat] = newval
                            features_to_delete.append(feature)
                            #print(feature_dico)
                        """
                            
                    for feature in new_feature_dico.keys():
                        feature_dico[feature] = new_feature_dico[feature]
                        
                    
                    for feature in features_to_delete:
                        if feature in feature_dico.keys():
                            del feature_dico[feature]
                    #print(feature_dico)
                    
                        
                    #print(feature_dico)
                    if identifier in annotations.keys():
                        for item in annotations[identifier]:
                            #print(identifier, item)
                            feature_dico[item] = annotations[identifier][item]
                        
                        if "Syl1" in feature_dico.keys() and feature_dico["Syl1"] == "FUSED":
                            #print("hi")
                            if "Syl1Duration" in feature_dico.keys():
                                del feature_dico["Syl1Duration"]
                            if "Syl1MeanF0" in feature_dico.keys():
                                del feature_dico["Syl1MeanF0"]
                            if "Syl1SemitonesFromUtteranceMean" in feature_dico.keys():
                                del feature_dico["Syl1SemitonesFromUtteranceMean"]
                        #print(identifier)
                        #print(feature_string)
                        feature_string = dico2FeatureString(feature_dico)
                        tree[token]['misc'] = feature_string
                    
                    """
                    identifier = str(treei+1)+":"+str(token)
                    misc_features = tree[token]['misc']
                    
                    if identifier in annotations.keys():              
                        for item in annotations[identifier]:
                            misc_features = misc_features + "|"
                            featstring = item+"="+annotations[identifier][item]
                            misc_features = misc_features + featstring
                    tree[token]['misc'] = misc_features
                    #print(tree[token]['misc'])
                    """
                    
            conll.trees2conllFile(trees, conll_outfolder+os.path.basename(infile))

print('done')